In [1]:
import mvp
import pandas as pd
import numpy as np

In [2]:
DB_PATH = '/home/naga/FintelligenceData/MetaTrader_M1.db'

In [3]:
petr3_raw = mvp.rawdata.RawData('PETR3',DB_PATH)

In [4]:
petr3 = mvp.curated.CuratedData(petr3_raw,{"MA":[500,5000]})

In [5]:
parameters = {'ModelParameters':{'MA':[500,1000]}, 'OperationParameters':{'SL': 3, 'TP': 4, 'IH': 100}}

In [6]:
model = mvp.primary.PrimaryModel(petr3_raw,'crossing-MA',parameters)

In [7]:
delta_MA = pd.DataFrame()
delta_MA['Delta']= (model.feature_data.df_curated['MA_500'] - model.feature_data.df_curated['MA_1000']).dropna()

In [10]:
def sign(x):
    if x>=0:
        return 1
    else:
        return -1

In [20]:
new_df = delta_MA['Delta'].apply(sign)
new_df = new_df.rolling(window=2).apply(lambda x: -1 if x[0]>x[1] else (1 if x[0]<x[1] else np.nan)  ,raw=True).dropna()

In [21]:
model.feature_data.df_curated['Delta'] = new_df

In [22]:
final_df = model.feature_data.df_curated.drop(columns=['Open','High','Low','TickVol','Volume'])

In [23]:
final_df.dropna()

,Close,MA_500,MA_1000,Delta
DateTime,,,,
2015-12-04 15:00:00,8.85,9.28384,9.28390,-1.0
2015-12-09 13:26:00,9.28,8.49476,8.49314,1.0
2015-12-11 12:14:00,9.06,8.93374,8.93390,-1.0
2015-12-15 14:36:00,8.69,8.61922,8.61885,1.0
2015-12-18 11:58:00,8.35,8.68650,8.68695,-1.0
...,...,...,...,...
2020-11-19 14:48:00,24.29,24.46556,24.46559,-1.0
2020-11-20 16:08:00,24.38,24.43548,24.43509,1.0
2020-11-26 10:52:00,26.11,26.60264,26.60307,-1.0
